In [166]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **10Ex**

In [167]:
trainp='/kaggle/input/cifar10/cifar10/train/'
testp='/kaggle/input/cifar10/cifar10/test/'

In [168]:
os.listdir(trainp)

In [169]:
from PIL import Image
import matplotlib.pyplot as plt
a=Image.open(trainp+ 'frog/33492_frog.png')
plt.imshow(a)

# **1. 데이터 불러오기**

In [170]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
igen=ImageDataGenerator(rescale=1./255, validation_split=0.1)

In [171]:
bs=8
isize=(32,32)
ishape=(32,32,3)

In [172]:
traind=igen.flow_from_directory(trainp, target_size =(32,32), 
                               batch_size = bs, class_mode = 'categorical', subset='training')

In [173]:
vald=igen.flow_from_directory(trainp, target_size =(32,32), 
                               batch_size = bs, class_mode = 'categorical', subset='validation')

In [174]:
testd=igen.flow_from_directory(testp, target_size =(32,32), 
                               batch_size = bs, class_mode = 'categorical')

In [175]:
x, y=traind.next() #batch size만큼 이미지를 제너레이트
print(x.shape, y.shape)
for i in range(bs):
    plt.imshow((x[i]*255).astype('int'))
    plt.show()
    print(y[i])

lable: one-hot vector

문제분류: 10개의 class를 갖는 분류문제 activation='softmax', loss: categorical_crossentropy

# **2. 데이터 전처리**

ImageDataGenerator(rescale=1/255) 단계에서 완료하였다.

# **3. CNN모델 라이브러리 불러오기**

In [176]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# **4. CNN모델설정**

In [177]:
CNN = Sequential()
CNN.add(Conv2D(8, (3,3), activation='relu', input_shape=(32,32,3,)))
CNN.add(Conv2D(16, (3,3), activation='relu'))
CNN.add(MaxPooling2D())
#CNN.add(Flatten())
CNN.add(Conv2D(16,(3,3), activation='relu'))
CNN.add(BatchNormalization())
        
CNN.add(Conv2D(8, (3,3), activation='relu'))
CNN.add(BatchNormalization())
#CNN.add(MaxPooling2D())
CNN.add(Conv2D(32, (3,3), activation='relu'))
## CNN.add(Conv2D(32, (3,3))
CNN.add(MaxPooling2D())
CNN.add(Flatten())
CNN.add(Dense(64, activation='relu'))
CNN.add(Dropout(0.3))
CNN.add(Dense(32, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(16, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(10, activation='softmax'))
CNN.summary()

In [178]:
from tensorflow.keras.utils import plot_model
plot_model(CNN,'CNN@.png')

# **5. 모델 COmpile**

In [179]:
CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics = 'acc')


# **6. 모델 Fit**

In [180]:
hist = CNN.fit(traind, epochs=60, validation_data=vald)

# **7. loss와 val_loss 그래프 그리기**

In [181]:
loss=hist.history['loss']
val_loss=hist.history['val_loss']
acc=hist.history['acc']
val_acc=hist.history['val_acc']
ep=np.arange(1,len(loss)+1)

In [182]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
plt.plot(ep, loss, 'b', label='loss')
plt.plot(ep, val_loss, 'g', label="val_loss")
plt.title('loss')
plt.legend()

# **8. acc와 val_acc 그래프 그리기**

In [183]:
plt.figure(figsize=(10,8))
plt.plot(ep, acc, 'b', label='acc')
plt.plot(ep, val_acc, 'g', label="val_acc")
plt.title('Accuracy')
plt.legend()

# **9. test_data evaluation**

In [184]:
CNN.evaluate(testd)